# **base GRN - *from atlas of ATAC regulatory regions***

In [2]:
import pandas as pd

In [3]:
cd /home/jovyan/jm_jlab/data_indNeuro/consensus_atlas_ATACregions_hg38/GREAT_results/

/home/jovyan/jm_jlab/data_indNeuro/consensus_atlas_ATACregions_hg38/GREAT_results


In [6]:
great_output  = pd.read_csv("./consensus_peaks_hg38_GREAT.txt", sep='\t')

In [7]:
len(great_output)

392961

In [8]:
great_output.head()

,# GREAT version 4.0.4,Species assembly: hg38,"Association rule: Basal+extension: 5000 bp upstream, 1000 bp downstream, 1000000 bp max extension, curated regulatory domains included"
0,tr21singleome_0,OR4F5 (-55141),NaN
1,tr21singleome_1,"OR4F5 (+115478), OR4F29 (+270800)",NaN
2,tr21singleome_2,"OR4F5 (+116065), OR4F29 (+270213)",NaN
3,tr21singleome_3,"OR4F5 (+126049), OR4F29 (+260229)",NaN
4,tr21singleome_8,"OR4F29 (-182330), OR4F16 (+52646)",NaN


In [9]:
great_output = great_output.iloc[:,0:2]
great_output.rename(columns={'# GREAT version 4.0.4':"peak_name", 'Species assembly: hg38':'gene_short_name'}, inplace=True)

In [10]:
len(great_output)

392961

In [11]:
#Cleaning output
great_output.head()

,peak_name,gene_short_name
0,tr21singleome_0,OR4F5 (-55141)
1,tr21singleome_1,"OR4F5 (+115478), OR4F29 (+270800)"
2,tr21singleome_2,"OR4F5 (+116065), OR4F29 (+270213)"
3,tr21singleome_3,"OR4F5 (+126049), OR4F29 (+260229)"
4,tr21singleome_8,"OR4F29 (-182330), OR4F16 (+52646)"


In [12]:
#Cleaning output
great_output.tail()

,peak_name,gene_short_name
392956,mk20_103807,NLGN4Y (+805180)
392957,mk20_103808,NLGN4Y (+956846)
392958,mk20_103809,NONE
392959,mk20_103810,"HSFY2 (-130682), KDM5D (+840506)"
392960,mk20_103811,"HSFY2 (-224311), KDM5D (+746877)"


In [13]:
great_output['gene_short_name'] = great_output['gene_short_name'].str.replace(r"\(.*?\)", "", regex=True).str.strip().str.split(" , ")

In [14]:
great_output = great_output.explode('gene_short_name')

In [15]:
great_output = great_output[great_output["gene_short_name"].str.contains("NONE")==False]

In [16]:
print(great_output.head())
print(len(great_output))

         peak_name gene_short_name
0  tr21singleome_0           OR4F5
1  tr21singleome_1           OR4F5
1  tr21singleome_1          OR4F29
2  tr21singleome_2           OR4F5
2  tr21singleome_2          OR4F29
679729


In [19]:
all_sorted = pd.read_csv("/home/jovyan/jm_jlab/data_indNeuro/consensus_atlas_ATACregions_hg38/consensus_signals.bed", sep='\t', header=None)

In [20]:
all_sorted.head(2)

,0,1,2,3
0,chr1,10027,10528,tr21singleome_0
1,chr1,180646,181147,tr21singleome_1


In [21]:
all_sorted.rename(columns={0:'chr',1:'start',2:'end',3:'peak_name'}, inplace=True)

In [22]:
great_output = great_output.merge(all_sorted, on = ['peak_name'])

In [23]:
great_output.head(2)

,peak_name,gene_short_name,chr,start,end
0,tr21singleome_0,OR4F5,chr1,10027,10528
1,tr21singleome_1,OR4F5,chr1,180646,181147


In [24]:
data = {'peak_id': great_output.iloc[:,2].astype(str)+"_"+great_output.iloc[:,3].astype(str)+"_"+great_output.iloc[:,4].astype(str), 
 'gene_short_name': great_output.iloc[:,1]}

In [25]:
annotated = pd.DataFrame(data)

In [26]:
annotated.head(2)

,peak_id,gene_short_name
0,chr1_10027_10528,OR4F5
1,chr1_180646_181147,OR4F5


In [27]:
print(len(annotated))

679729


# **Scan for TF binding motifs** - CELLORACLE

**Scan for TF binding motifs** - CELLORACLE

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

import celloracle as co
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object
co.__version__

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


'0.10.14'

In [29]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

In [34]:
if not genome_installation:
    import genomepy
    genomepy.install_genome(name=ref_genome, provider="UCSC")
else:
    print(ref_genome, "is installed.")

In [35]:
def decompose_chrstr(peak_str):
    """
    Args:
        peak_str (str): peak_str. e.g. 'chr1_3094484_3095479'

    Returns:
        tuple: chromosome name, start position, end position
    """

    *chr_, start, end = peak_str.split("_")
    chr_ = "_".join(chr_)
    return chr_, start, end

from genomepy import Genome

def check_peak_format(peaks_df, ref_genome):
    """
    Check peak format.
     (1) Check chromosome name.
     (2) Check peak size (length) and remove sort DNA sequences (<5bp)

    """

    df = peaks_df.copy()

    n_peaks_before = df.shape[0]

    # Decompose peaks and make df
    decomposed = [decompose_chrstr(peak_str) for peak_str in df["peak_id"]]
    df_decomposed = pd.DataFrame(np.array(decomposed), index=peaks_df.index)
    df_decomposed.columns = ["chr", "start", "end"]
    df_decomposed["start"] = df_decomposed["start"].astype(int)
    df_decomposed["end"] = df_decomposed["end"].astype(int)

    # Load genome data
    genome_data = Genome(ref_genome)
    all_chr_list = list(genome_data.keys())


    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])


    # Filter peaks with invalid chromosome name
    n_threshold = 5
    df = df[(lengths >= n_threshold) & df_decomposed.chr.isin(all_chr_list)]

    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])

    # Data counting
    n_invalid_length = len(lengths[lengths < n_threshold])
    n_peaks_invalid_chr = n_peaks_before - df_decomposed.chr.isin(all_chr_list).sum()
    n_peaks_after = df.shape[0]


    #
    print("Peaks before filtering: ", n_peaks_before)
    print("Peaks with invalid chr_name: ", n_peaks_invalid_chr)
    print("Peaks with invalid length: ", n_invalid_length)
    print("Peaks after filtering: ", n_peaks_after)

    return df

In [36]:
peaks = check_peak_format(annotated, ref_genome)

Peaks before filtering:  679729
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  679729


In [37]:
peaks.head()

,peak_id,gene_short_name
0,chr1_10027_10528,OR4F5
1,chr1_180646_181147,OR4F5
2,chr1_180646_181147,OR4F29
3,chr1_181233_181734,OR4F5
4,chr1_181233_181734,OR4F29


In [38]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks,
                ref_genome=ref_genome)

In [39]:
import os, glob
from gimmemotifs.motif import MotifConfig
config = MotifConfig()
motif_dir = config.get_motif_dir()

# Get motif data names
motifs_data_name = [i for i in os.listdir(motif_dir) if i.endswith(".pfm")]
motifs_data_name.sort()
motifs_data_name

['CIS-BP.pfm',
 'ENCODE.pfm',
 'HOCOMOCOv10_HUMAN.pfm',
 'HOCOMOCOv10_MOUSE.pfm',
 'HOCOMOCOv11_HUMAN.pfm',
 'HOCOMOCOv11_MOUSE.pfm',
 'HOMER.pfm',
 'IMAGE.pfm',
 'JASPAR2018.pfm',
 'JASPAR2018_fungi.pfm',
 'JASPAR2018_insects.pfm',
 'JASPAR2018_nematodes.pfm',
 'JASPAR2018_plants.pfm',
 'JASPAR2018_urochordates.pfm',
 'JASPAR2018_vertebrates.pfm',
 'JASPAR2020.pfm',
 'JASPAR2020_fungi.pfm',
 'JASPAR2020_insects.pfm',
 'JASPAR2020_nematodes.pfm',
 'JASPAR2020_plants.pfm',
 'JASPAR2020_urochordates.pfm',
 'JASPAR2020_vertebrates.pfm',
 'JASPAR2022.pfm',
 'JASPAR2022_fungi.pfm',
 'JASPAR2022_insects.pfm',
 'JASPAR2022_nematodes.pfm',
 'JASPAR2022_plants.pfm',
 'JASPAR2022_urochordates.pfm',
 'JASPAR2022_vertebrates.pfm',
 'RSAT_insects.pfm',
 'RSAT_plants.pfm',
 'RSAT_vertebrates.pfm',
 'SwissRegulon.pfm',
 'factorbook.pfm',
 'gimme.vertebrate.v5.0.pfm']

**Homococo**

In [41]:
# You can load motif files with "read_motifs"
from gimmemotifs.motif import read_motifs

path = os.path.join(motif_dir, "HOCOMOCOv11_HUMAN.pfm")
motifs = read_motifs(path)

# Check first 10 motifs
motifs[:10]

[AHR_HUMAN.H11MO.0.B_nTnGCGTGn,
 AIRE_HUMAN.H11MO.0.C_nTTGGwwnwAwTGGnTwn,
 ALX1_HUMAN.H11MO.0.B_nTAATTrnATTA,
 ANDR_HUMAN.H11MO.0.A_TGTnCTnnnnTGTTTryw,
 AP2A_HUMAN.H11MO.0.A_nTnsCCTGrGGsnAn,
 AP2B_HUMAN.H11MO.0.B_GCCyGnGGGn,
 AP2C_HUMAN.H11MO.0.A_nTGsCCTGrGGsnn,
 ARI5B_HUMAN.H11MO.0.C_nnyknGTATTskn,
 ARNT_HUMAN.H11MO.0.B_nTACGTGmC,
 ASCL1_HUMAN.H11MO.0.A_nnnCAsCTGCynyn]

In [42]:
tfi.scan(motifs=motifs, verbose=True) #error with HOMOCOCO v11

Checking your motifs... Motifs format looks good. 

Initiating scanner... 

Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 

Motif scan started .. It may take long time.



scanning:   0%|          | 0/385069 [00:00<?, ? sequences/s]

In [43]:
# Check motif scan results
tfi.scanned_df.head()

,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100006240_100006741,AHR_HUMAN.H11MO.0.B,AHR,,7.519250,81,-1
1,chr10_100006240_100006741,AIRE_HUMAN.H11MO.0.C,AIRE,,9.127656,292,-1
2,chr10_100006240_100006741,BACH1_HUMAN.H11MO.0.A,BACH1,,11.048207,241,-1
3,chr10_100006240_100006741,BACH2_HUMAN.H11MO.0.A,BACH2,,9.697703,243,-1
4,chr10_100006240_100006741,BACH2_HUMAN.H11MO.0.A,BACH2,,8.253610,9,-1


In [44]:
# Reset filtering
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

Filtering finished: 11696929 -> 3268833
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/378212 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/18193 [00:00<?, ?it/s]

  0%|          | 0/399 [00:00<?, ?it/s]

In [45]:
df = tfi.to_dataframe()
df.head()

,peak_id,gene_short_name,AHR,AIRE,ALX1,AR,ARID5B,ARNT,ARNTL,ASCL1,...,ZNF680,ZNF708,ZNF76,ZNF768,ZNF770,ZNF8,ZNF816,ZNF85,ZSCAN22,ZSCAN31
0,chr10_100006240_100006741,ABCC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chr10_100006240_100006741,DNMBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr10_100009160_100009661,DNMBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr10_100009709_100010210,DNMBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,chr10_100010303_100010804,DNMBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
pwd

'/home/jovyan/jm_jlab/tmp_folder3'

In [47]:
# Save result as a dataframe
df.to_parquet("/home/jovyan/jm_jlab/data_indNeuro/consensus_atlas_ATACregions_hg38/base_GRN_dataframe_HOCOMOCOv11.parquet")